In [1]:
import torch
from torch import nn
from torch.nn import functional as F 

# print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
print(device)

cuda:0


In [2]:
from GCNet.GCNet import GCNet

net = GCNet(device)
print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in net.parameters()])))

net = net.to(device)
net.train()
optimizer = torch.optim.AdamW(net.parameters(), lr=0.01)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, init_lr, num_steps + 100,
#                                           pct_start=0.01, cycle_momentum=False, anneal_strategy='linear')
    


Number of model parameters: 2791905


In [3]:
from utils.stereo_datasets import fetch_dataset, DFC2019
from tqdm import tqdm
import numpy as np

train_loader = fetch_dataset(dataset_name = 'DFC2019', root = '/home/lab1/datasets/DFC2019_track2_grayscale_8bit',
                            batch_size = 2, resize = [512, 512], mode = 'training')
print(train_loader.__len__())

for i_batch, data_blob in enumerate(tqdm(train_loader)):
    imgl, imgr, disp, valid = [x.to(device) for x in data_blob]
    disp_predict = net(imgl, imgr, 0, 64)
    disp_predict = disp_predict.squeeze(1)
    print(disp_predict.shape)    
    print(disp.shape)
    print(valid.shape)
    # valid = disp != 0
    # print(valid.shape)
    print(disp_predict[valid])
    break

2038


  0%|          | 0/2038 [00:01<?, ?it/s]

torch.Size([2, 512, 512])
torch.Size([2, 512, 512])
torch.Size([2, 512, 512])
tensor([31.0578, 32.1045, 30.5283,  ..., 31.3817, 30.0077, 33.4769],
       device='cuda:0', grad_fn=<IndexBackward0>)


In [ ]:
class SmoothL1Loss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, disp1,  target):
        loss1 = F.smooth_l1_loss(disp1, target)
        return loss1

In [ ]:
criterion = SmoothL1Loss().to(device)

optimizer.zero_grad()


In [ ]:
loss = criterion(disp_predict, disp)


In [ ]:
loss.backward()


In [ ]:
optimizer.step()

In [19]:
from PIL import Image
import numpy as np
from torchvision import transforms

filename = '/home/lab1/datasets/DFC2019_track2_grayscale_8bit/Track2-RGB-1/JAX_068_001_012_RIGHT_RGB.tif'
disp = Image.open(filename)
w, h = disp.size


1024 1024
(512, 512, 3)
tensor([[[ 0.2392,  0.1843,  0.1608,  ..., -0.3412, -0.3255, -0.2863],
         [ 0.4039,  0.3725,  0.3412,  ..., -0.2471, -0.3176, -0.3333],
         [ 0.4275,  0.4196,  0.3961,  ..., -0.2078, -0.2784, -0.3490],
         ...,
         [-0.0667, -0.0824, -0.0745,  ..., -0.7255, -0.7412, -0.6784],
         [ 0.0353, -0.0118, -0.0588,  ..., -0.6706, -0.7098, -0.7490],
         [ 0.2000,  0.1216,  0.0431,  ..., -0.6157, -0.4431, -0.4667]],

        [[ 0.2392,  0.1843,  0.1608,  ..., -0.3412, -0.3255, -0.2863],
         [ 0.4039,  0.3725,  0.3412,  ..., -0.2471, -0.3176, -0.3333],
         [ 0.4275,  0.4196,  0.3961,  ..., -0.2078, -0.2784, -0.3490],
         ...,
         [-0.0667, -0.0824, -0.0745,  ..., -0.7255, -0.7412, -0.6784],
         [ 0.0353, -0.0118, -0.0588,  ..., -0.6706, -0.7098, -0.7490],
         [ 0.2000,  0.1216,  0.0431,  ..., -0.6157, -0.4431, -0.4667]],

        [[ 0.2392,  0.1843,  0.1608,  ..., -0.3412, -0.3255, -0.2863],
         [ 0.4039,  0